In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key
# Read csv file
crime_data_df = pd.read_csv("../Resources/hate_crime.csv", low_memory=False)
# Define a function to be used later for making bar charts
def make_bar():
    plt.figure(figsize=(15, 8))
    color = mpl.cm.cool(np.linspace(.1, 1, 31))
    plt.rcParams['axes.facecolor'] = 'lightcyan'
    plt.bar(x, y, align='center',hatch=h,color=color, edgecolor="black", width= .5, linewidth=2.5)
    
    plt.xticks(x, rotation='vertical')
    plt.title(title, fontsize = 17)
    plt.xlabel(xl, fontsize = 17)
    plt.ylabel(yl, fontsize = 17) 
 
# Define a function to be used later for making pie charts
def make_pie():
    fig = plt.figure()
    fig.patch.set_facecolor('lightcyan')
    global e
    labels = x.index.values 
  
    colors = ['#33F6FF','#33A5FF','#8033FF','#C733FF','#F633FF']
   
    wp = { 'linewidth' : 1, 'edgecolor' : "black"}  
    
    plt.pie(x, labels=labels, colors=colors,wedgeprops = wp, explode=e, \
        autopct='%1.1f%%',pctdistance=0.79,startangle=90,shadow=True, radius=1.8 *1000,labeldistance=1.05)
    plt.title(title, fontsize = 17)
    plt.axis('equal')
    plt.legend(legend, loc="lower left",facecolor='white', framealpha=1)
    plt.tight_layout()   
# Define a function to reindex the months in order from Jan - Dec
def rdex1():
    global recount1
    recount1 = counts.reindex(index=["JAN",'FEB','MAR', 'APR','MAY','JUN','JUL','AUG', 'SEP','OCT','NOV','DEC'])

# Define a function to reindex the days in order from 1 - 31
def rdex2():
    global recount2
    recount2 = counts.reindex(index=["01",'02','03', '04','05','06','07','08', '09','10','11','12',"13",'14','15',\
                                '16','17','18','19','20', '21','22','23','24','25','26','27','28','29','30','31'])
 def api_query():   
    # Gathering LAT and LNG for each state to use in map
    state_unique = peak_year["STATE_NAME"].unique()
    city_overall = top_5_overall.index.values
    city_2001 = top_5_2001.index.values

    # Set the API base URL
    url = "https://api.geoapify.com/v1/geocode/search?country=United%20States%20of%20America"

    # Define empty lists for the data of each state and desired cities,set them to global
    # to be used outside of the function, and added record counter
    global state_data
    global city_overall_data
    global city_2001_data
    
    state_data = []
    city_overall_data = []
    city_2001_data = []
    rcnt = 1

    # Print to logger
    print("Beginning State Data Retrieval     ")
    print("-----------------------------")

    # Loop through all the cities in our list to fetch location data
    for state in state_unique:

        # Create endpoint URL with each city
        state_url = url + "&state="+ state + "&apiKey=" + geoapify_key

    #     print(f"Processing Record{rcnt} for {state}")
    #     rcnt += 1
        # Run an API request for each of the cities
        try:
            # Parse the JSON and retrieve data
            state_info = requests.get(state_url).json()

            # Parse out latitude, longitude
            state_lat = state_info["features"][0]["properties"]["lat"]
            state_lng = state_info["features"][0]["properties"]["lon"]

            # Append the state information into state_data list
            state_data.append({"State": state, 
                              "Lat": state_lat, 
                              "Lng": state_lng})
    # If an error is experienced, skip the city
        except:
            pass
            print("you screwed up somewhere")
    print("-----------------------------")
    print("State Data Retrieval Complete\nBeginning Overall City Data Retrieval")
    print("-----------------------------")
    rcnt = 1

    for city in city_overall:


        # Create endpoint URL with each city
        city_url = url + "&city="+ city + "&apiKey=" + geoapify_key

        print(f"Processing Record {rcnt} for {city}")
        rcnt += 1
        # Run an API request for each of the cities
        try:
            # Parse the JSON and retrieve data
            city_info = requests.get(city_url).json()

            # Parse out latitude, longitude
            city_lat = city_info["features"][0]["properties"]["lat"]
            city_lng = city_info["features"][0]["properties"]["lon"]

            # Append the state information into state_data list
            city_overall_data.append({
                              "City": city, 
                              "Lat": city_lat, 
                              "Lng": city_lng})
    # If an error is experienced, skip the city
        except:
            pass
            print("you screwed up somewhere")
    print("-----------------------------")
    print("Overall City Data Retrieval Complete\nBeginning 2001 City Data Retrieval")
    print("-----------------------------")
    rcnt = 1

    for city in city_2001:


        # Create endpoint URL with each city
        city_url = url + "&city="+ city + "&apiKey=" + geoapify_key

        print(f"Processing Record {rcnt} for {city}")
        rcnt += 1
        # Run an API request for each of the cities
        try:
            # Parse the JSON and retrieve data
            city_info = requests.get(city_url).json()

            # Parse out latitude, longitude
            city_lat = city_info["features"][0]["properties"]["lat"]
            city_lng = city_info["features"][0]["properties"]["lon"]

            # Append the state information into state_data list
            city_2001_data.append({
                              "City": city, 
                              "Lat": city_lat, 
                              "Lng": city_lng})
    # If an error is experienced, skip the city
        except:
            pass
            print("you screwed up somewhere")

    # Indicate that Data Loading is complete 
    print("-----------------------------")
    print("2001 City Data Retrieval Complete")
    print("-----------------------------")
    print("Your Geoapify Requests Are Complete")
    print("-----------------------------")


# GROUP 3 PROJECT


### For this project, will be looking at the FBI Hate Crimes dataset from 1991 to 2018. We will look at the raw data and remove anything that could skew our results that we hope to find. We will determine the offense that occurs the most throughout the 28 year span, then compare it to the top 5 cities with the most hate crime incidents. We also will determine which year had the most offenses and compare the same offense to the top 5 cities of the peak crime year. We want to compare the bias description (targeted group) to the offender race and compare that to the offense type. We will plot the most common crimes and use that to compare over the years. By the end of our analysis, we will find out which state and city within that state which has the highest hate crime. 

# Display the dataframe
crime_data_df.head(3)
# Clean up the data
# Drop columns of data not needed
clean_crime = crime_data_df.drop(columns=["ORI", "PUB_AGENCY_UNIT", "JUVENILE_VICTIM_COUNT","ADULT_VICTIM_COUNT", "ADULT_OFFENDER_COUNT", 'JUVENILE_OFFENDER_COUNT', "POPULATION_GROUP_CODE", "POPULATION_GROUP_DESC", "LOCATION_NAME", "DIVISION_NAME"])

# Dropping "Guam", "Federal", "District of Columbia" from clean_crime to keep only the named continental states
state_drop = ["Guam", "Federal", "District of Columbia"]
clean_crime = clean_crime[~clean_crime["STATE_NAME"].isin(state_drop)]

#Split Ofense names into main and sub categories to only look at the main offense
clean_crime[["MAIN_OFFENSE","SUB_OFFENSE"]] = clean_crime["OFFENSE_NAME"].str.split('/', n=1, expand= True)
clean_crime["MAIN_OFFENSE"] = clean_crime["MAIN_OFFENSE"].str.split(';', n=1, expand= True)

# Split the incident date into year,month, and day
dates = clean_crime['INCIDENT_DATE'].str.split("-", n = 2, expand = True) 

# Add new columns to the cleaned dataframe and droped the original date column
clean_crime["DAY"]= dates[0]
clean_crime["MONTH"]= dates[1]
clean_crime["LAT"] = ""
clean_crime["LNG"] = ""
clean_crime.drop(columns =["INCIDENT_DATE"], inplace = True)

# Rename the year column and replace offender race to only the main category and change the order of the columns to have all 3 date columns together
clean_crime.rename(columns={'DATA_YEAR': 'YEAR'}, inplace=True)
clean_crime['OFFENDER_RACE'] = clean_crime['OFFENDER_RACE'].replace(['American Indian or Alaska Native', 'Asian' , 'Native Hawaiian or Other Pacific Islander'],'AAPI').copy()
clean_crime= clean_crime[['STATE_NAME','STATE_ABBR','PUB_AGENCY_NAME','LAT','LNG','YEAR',"MONTH","DAY",'MAIN_OFFENSE','BIAS_DESC','OFFENDER_RACE','VICTIM_COUNT']]




#display new dataframe
# clean_crime
# Make a pie chart showing the top 5 most occuring crimes
crime_type = clean_crime['MAIN_OFFENSE'].value_counts().nlargest(5)
x = crime_type
labels = None
legend = crime_type.index.values
title= ("Most Common Offense")
e = (100, 30, 30, 30, 30)
make_pie()

#save to Output folder
plt.savefig("../Output/Fig11.jpeg")

#Display the chart
plt.show()
# Which hate crime most prevalent?
# Getting the counts of the 5 most prevalent ofense types
offense_info= clean_crime['MAIN_OFFENSE'].value_counts().nlargest(n=5)
offense_info
# Getting the counts of the most prevalent offender types
offender_info= clean_crime['OFFENDER_RACE'].value_counts().nlargest(n=5)
offender_info
# What bias description (targeted group) is most susceptible?
# Getting the counts of the most prevalent Bias Description
bias_info = clean_crime['BIAS_DESC'].value_counts().nlargest(n=5)
bias_info
# Show the counts of the most prevalent Bias Description in a bar chart 
x = bias_info.index.values
y = bias_info.values
h = ''
title="Bias Description of the victims\nof Hate Crimes over 28 Years"
xl = 'Bias Description'
yl = 'Number of Hate Crimes in The US'
legend = bias_info.index
make_bar()

#save to Output folder
plt.savefig("../Output/Fig8.jpeg")

#Display the chart
plt.show()
# Graph a nested pie chart showing the offenders off the top bias description
# Rename Dataframe to include only the needed columns
off_bias_df = clean_crime[clean_crime['BIAS_DESC'] == "Anti-Black or African American" ]
pie_df = off_bias_df[['BIAS_DESC', 'OFFENDER_RACE','VICTIM_COUNT',]]

# Set inner and outer counts and labels
outer = pie_df.groupby('OFFENDER_RACE').sum()
outer.index

inner = pie_df.groupby(['BIAS_DESC',]).sum()
inner_labels = inner.index.get_level_values(0)

# Set parameters and plots
fig, ax = plt.subplots(figsize=(24,12))
size = 0.3

fig.patch.set_facecolor('lightcyan')
colors = ['#F633FF','#8033FF','#33F6FF','#C733FF','#33A5FF']
ax.pie(outer.values.flatten(), radius=1,
       labels=outer.index.values,
       colors = ['#33F6FF','#33A5FF','#8033FF','#C733FF','#F633FF'],
       autopct='%1.1f%%',
       pctdistance=0.79,
       wedgeprops=dict(width=size, edgecolor='w'))

ax.pie(inner.values.flatten(), radius=1-size, 
       labels = inner_labels,
       colors = ['#2eaeb3'],
       labeldistance= .1,
       wedgeprops=dict(width=size, edgecolor='w'))

# Add title and legend
ax.set(aspect="equal", title='Offender Totals for the most Suseptible\nBias Description (Anti-Black or African American)', )
legend = outer.index.values
plt.legend(legend, loc="upper left",facecolor='white', framealpha=1)

#save to Output folder
plt.savefig("../Output/Fig12.jpeg")

#Display the chart
plt.show()
# What year has the most crimes throughout the years?
# Get the count of offenses for each year and plot a bar chart 
year_counts = clean_crime['YEAR'].value_counts()
x = year_counts.index.values
y = year_counts.values
h = ''
title="Hate Crime Counts Over 28 Years"
xl = 'Year'
yl = 'Number of Hate Crimes in The US'
make_bar()

#save to Output folder
plt.savefig("../Output/Fig1.jpeg")

#Display the chart
plt.show()
# Which city/state has most hate crime(s)?
# Get the count of offenses for the peak year and plot a bar chart 
peak_year = clean_crime[clean_crime['YEAR'] == 2001]
states = peak_year['STATE_NAME'].value_counts().nlargest(15)
x = states.index.values
y = states.values
h = ''
title = "Peak Year Hate Crimes by The Top 15\n States With The Most Hate Crimes"
xl = '2001'
yl = 'Number of Hate Crimes'
make_bar()

# Save to Output folder
plt.savefig("../Output/Fig2.jpeg")

# Display the chart
plt.show()
# Plot a line chart showing the hate crime trend during the peak crime year
peak_year_df = peak_year['MONTH'].value_counts()

# Reorganize the index
rdex = peak_year_df.reindex(index=["JAN",'FEB','MAR', 'APR','MAY','JUN','JUL','AUG', 'SEP','OCT','NOV','DEC'])
x= rdex.index.values
y= rdex.values
plt.rcParams['figure.figsize'] = [15, 8]
plt.plot(x, y, marker='x')
plt.xlabel("Month", fontsize=17)
plt.ylabel("Number of Incidents", fontsize=17)
plt.title("Number of Hate Crimes in 2001 by Month", fontsize=17)
plt.xticks(x)

# Save to Output folder
plt.savefig("../Output/Fig9.jpeg")

# Display the chart
plt.show()
# Plot a line chart showing the hate crime trend during the peak crime month
peak_day = peak_year[peak_year['MONTH'] == 'SEP']
peak_day_df = peak_day['DAY'].value_counts()

# Reorganize the index
ddex = peak_day_df.reindex(index=["01",'02','03', '04','05','06','07','08', '09','10','11','12',"13",'14','15',\
                                '16','17','18','19','20', '21','22','23','24','25','26','27','28','29','30','31'])
z = ddex.index.values
h =ddex.values
plt.plot(z,h, marker="o")
plt.ylabel("Number of Incidents")
plt.title("Number of Hate Crimes in 2001 by day in September", fontsize=17)
plt.xlabel("Day", fontsize=17)
plt.ylabel("Number of Incidents", fontsize=17)
plt.xticks(z)

# Save to Output folder
plt.savefig("../Output/Fig10.jpeg")

# Display the chart
plt.show()
# Get the count of offenses for the state in the peak year with the most offenses 
max_state_df = peak_year[peak_year['STATE_NAME'] == "California"]
bias = max_state_df['BIAS_DESC'].value_counts()
# bias
# Plot a bar chart for the state with the most offenses using the defined function
x = bias.index.values
y = bias.values
h = ''
title = "Across California 2001"
xl = 'Bias Description'
yl = 'Count of Offenses'
make_bar()

# Save to Output folder
plt.savefig("../Output/Fig3.jpeg")

# Display the chart
plt.show()
# Plot a bar chart for the state with the most offenses broken down by month using the defined function
counts = max_state_df['MONTH'].value_counts()
rdex1()
x = recount1.index.values
y = recount1.values
title = "California 2001"
xl = 'Month'
yl = 'Offense Counts'
h = ""
make_bar()

# Save to Output folder
plt.savefig("../Output/Fig4.jpeg")

# Display the chart
plt.show()
# Plot a bar chart for the days in September using the defined function and reset the index chronologically
counts = max_state_df['DAY'].value_counts()
rdex2()
x = recount2.index.values
y = recount2.values
title  = 'September California 2001'
xl ='Day in September'
yl = 'September Hate Crime Count'
h = ""
make_bar()

# Save to Output folder
plt.savefig("../Output/Fig5.jpeg")

# Display the chart
plt.show()
# Collect the data for the top 5 cities with the most hate crimes over the span of 28 years
cities_overall = clean_crime['PUB_AGENCY_NAME'].value_counts().nlargest(n=5)

# Change the collected data into a dataframe and rename the column name
top_5_overall = pd.DataFrame(cities_overall)
top_5_overall = top_5_overall.rename(columns={'PUB_AGENCY_NAME': 'OFFENSE_COUNT'})

# Collect the data for the top 5 cities with the most hate crimes in the peak year of 2001
cities_year = peak_year['PUB_AGENCY_NAME'].value_counts().nlargest(n=5)

# Change the collected data into a dataframe and rename the column name
top_5_2001 = pd.DataFrame(cities_year)
top_5_2001 = top_5_2001.rename(columns={'PUB_AGENCY_NAME': 'OFFENSE_COUNT'})
# using a function, get the Lat and Lon of the states and cities in question
api_query()
# Display top 5 cities across the years with the most hate crimes
top_5_overall
# Display top 5 cities across 2001 with the most hate crimes
top_5_2001
# Display appended data
city_2001_data
# Change the appended data into a dataframe and add the offense counts
city_data_01 = pd.DataFrame(city_2001_data)
city_data_01['OFFENSE_COUNT']= top_5_2001["OFFENSE_COUNT"].values
city_data_01
# Display appended data
city_overall_data
# Change the appended data into a dataframe and add the offense counts
city_data_all = pd.DataFrame(city_overall_data)
city_data_all['OFFENSE_COUNT']= top_5_overall["OFFENSE_COUNT"].values
city_data_all
# Display appended data
state_data
# Change the appended data into a dataframe 
state_data_df = pd.DataFrame(state_data)
state_data_df.head()

# Configure the map plot for the top 5 cities over all
map_it = city_data_all.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    size = 3000,
    color = "#F633FF",
    alpha = 0.75,
    colorbar=True,
    hover_cols = "",
    
   
    )

# city name, offense count
# Display the map
map_it
# Showing the top 5 cities across 28 years that have the most hate crimes
x = cities_overall
e = (50, 30, 30, 30, 30)
title = "Top 5 Cities from 1991 to 2018\n With the Most Hate Crimes"
legend = cities_overall.index.values
make_pie()

# Save to Output folder
plt.savefig("../Output/Fig6.jpeg")

# Display the chart
plt.show()
# Configure the map plot for the top 5 cities in 2001
map_it = city_data_01.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    size = 3000,
    color = "#33F6FF",
    alpha = 0.75,
   
    )

# Display the map
map_it

# Showing the top 5 cities in 2001 that have the most hate crimes
x = cities_year
e = (100, 30, 30, 30, 30)
title = "Top 5 Cities in 2001\n With the Most Hate Crimes"
make_pie()

# Save to Output folder
plt.savefig("../Output/Fig7.jpeg")

# Display the chart
plt.show()



,INCIDENT_ID,DATA_YEAR,ORI,PUB_AGENCY_NAME,PUB_AGENCY_UNIT,AGENCY_TYPE_NAME,STATE_ABBR,STATE_NAME,DIVISION_NAME,REGION_NAME,...,OFFENDER_RACE,OFFENDER_ETHNICITY,VICTIM_COUNT,OFFENSE_NAME,TOTAL_INDIVIDUAL_VICTIMS,LOCATION_NAME,BIAS_DESC,VICTIM_TYPES,MULTIPLE_OFFENSE,MULTIPLE_BIAS
0,3015,1991,AR0040200,Rogers,NaN,City,AR,Arkansas,West South Central,South,...,White,NaN,1,Intimidation,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-Black or African American,Individual,S,S
1,3016,1991,AR0290100,Hope,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,1,Simple Assault,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,S,S
2,43,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,1,Aggravated Assault,1.0,Residence/Home,Anti-Black or African American,Individual,S,S
3,44,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,2,Aggravated Assault;Destruction/Damage/Vandalis...,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,M,S
4,3017,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,1,Aggravated Assault,1.0,Service/Gas Station,Anti-White,Individual,S,S
